In [ ]:
 def hopcroft_karp(bipartite_matrix):
        
        def bfs(bipartite_matrix, matching):
            free_u = set(range(len(bipartite_matrix))) - {x[0] for x in matching}
            free_v = set(range(len(bipartite_matrix[0]))) - {x[1] for x in matching}
            visited_u = set()
            visited_v = set()
            q = list(zip(free_u, [0]*len(free_u)))
            k = 0
            set_v = set()
            while q:
                top, level = q.pop(0)
                if level%2==0:
                    if top not in visited_u:
                        visited_u.add(top)
                        q.append([(i, level+1) for i in range(len(bipartite_matrix[top])) if bipartite_matrix[top][i] and (top, i) not in matching])
                else:
                    if top not in visited_v:
                        visited_v.add(top)
                        q.append([(i, level+1) for i in range(len(bipartite_matrix)) if bipartite_matrix[i][top] and (i, top) in matching])
                        if top in free_v:
                            if k==0: k = level
                            if level==k: set_v.add(top)
                            else: break
                        else:
                            if k>0 and level>k: break
            return set_v
        
        def dfs(bipartite_matrix, matching, set_v):
            free_u = set(range(len(bipartite_matrix))) - {x[0] for x in matching}
            visited_u = set()
            visited_v = set()
            s = list(zip(set_v, [0]*len(set_v)))
            p = set()
            while s:
                top, level = s[-1]
                if level%2==0:
                    if top not in visited_v:
                        visited_v.add(top)
                        to_append = [(i, level+1) for i in range(len(bipartite_matrix)) if bipartite_matrix[i][top] and (i, top) in matching]
                        if to_append: s.append(to_append)
                    else: s.pop()
                else:
                    if top not in visited_u:
                        visited_u.add(top)
                        if top in free_u:
                            if k
                        to_append = [(i, level+1) for i in range(len(bipartite_matrix[top])) if bipartite_matrix[top][i] and (top, i) not in matching and i not in visited_v]
                        if to_append: s.append(to_append)
                    else: s.pop()
                    
                    
                
            
        paths = set([len(bipartite_matrix)])
        matchings = set()
        
        while paths:
            
   

In [458]:
from collections.abc import MutableSequence

class EgocentricSBM(MutableSequence):
    
    def __init__(self, num_blocks, dim_names=None, model_name='egocentric_sbm'):
        
        if not self.iterable(num_blocks): num_blocks = (num_blocks,)
        num_blocks = tuple(num_blocks)
        if not all([isinstance(b, int) and b>1 for b in num_blocks]): raise ValueError('number of blocks must be an integer greater than 1')
        self._precision = 8
        self._curridx = 0
        self._currdim = 0
        self.ndim = len(num_blocks)
        self.shape = num_blocks
        self.name = str(model_name)
        self.dims = tuple()
        self.dimsdict = dict()
        self.dimsfull = tuple()
        self.set_dims(dim_names)
        self.pi = tuple([(1/b,)*b for b in num_blocks])
        self.rho = tuple([(1/b,)*b for b in num_blocks])
        self.omega = tuple([(0.,)*b for b in num_blocks])
        self.check_consistency()
        
    @property
    def precision(self):
        return self._precision
    
    @precision.setter
    def precision(self, value):
        from warnings import warn
        warn('setting "precision" can affect model consistency checks', RuntimeWarning)
        self._precision = value
        
    def __str__(self):
        out = ['name:\t'+self.name]
        out.append('ndim:\t'+str(self.ndim))
        out.append('shape:\t'+str(self.shape))
        out.append('----\t------')
        out.append('dims\tblocks')
        out.append('----\t------')
        out += [str(x[0])+'\t'+str(x[1]) for x in self.dims]
        out.append('----\t------')
        out.append('pi:\t'+str(self.pi))
        out.append('rho:\t'+str(self.rho))
        out.append('omega:\t'+str(self.omega))
        return '\n'.join(out)
    
    def __len__(self):
        return self.ndim
    
    def __copy__(self):
        out = type(self)(self.shape, self.dims, self.name)
        out.pi = self.pi
        out.rho = self.rho
        out.omega = self.omega
        out._precision = self._precision
        out._currdim = self._currdim
        return out
    
    def copy(self):
        return self.__copy__()
    
    def sort(self, inplace=True, reverse=True, blocks=True, dims=True):
        idx_blocks = list()
        if blocks or dims:
            # sorts by pi, followed by omega to break ties, followed by rho to break ties
            for i in range(self.ndim):
                idx_blocks.append(tuple(sorted(zip(range(self.shape[i]), zip(self.pi[i], self.omega[i], self.rho[i])), key=lambda t: t[1], reverse=reverse)))
        idx_dims = list()
        if dims:
            from itertools import chain
            # sorts by number of blocks, followed by atts of block with largest pi, and so on
            temp = [(self.shape[i],) + tuple(chain(*[x[1] for x in idx_blocks[i]])) for i in range(self.ndim)]
            idx_dims = sorted(zip(range(self.ndim), temp), key=lambda t: t[1], reverse=reverse)
        if inplace:
            if dims:
                self.shape = tuple([x[1][0] for x in idx_dims])
                if blocks: temp = [(self.dims[x[0]][0], [self.dims[x[0]][1][y[0]] for y in idx_blocks[x[0]]]) for x in idx_dims]
                else: temp = [self.dims[x[0]] for x in idx_dims]
                self.set_dims(temp)
            if blocks:
                self.pi = tuple([tuple([y[1][0] for y in idx_blocks[x[0]]]) for x in idx_dims])
                self.omega = tuple([tuple([y[1][1] for y in idx_blocks[x[0]]]) for x in idx_dims])
                self.rho = tuple([tuple([y[1][2] for y in idx_blocks[x[0]]]) for x in idx_dims])
        else:
            if dims: idx_dims = [x[0] for x in idx_dims]
            if blocks: idx_blocks = [[x[0] for x in y] for y in idx_blocks]
            return idx_dims, idx_blocks
    
    def get_key(self, key):
        if key in self.dimsdict:
            key = self.dimsdict[key]
        else:
            if isinstance(key, int):
                if key<0: key += self.ndim
                if key>=0 and key<self.ndim: pass
                else: raise IndexError('sbm index out of range; must be less than ndims')
            else: raise KeyError('sbm key "%s" not found; must be dimension name'%str(key))
        return key
    
    def insert(self, key, num_blocks, block_names=None):
        if not (isinstance(num_blocks, int) and num_blocks>1): raise ValueError('number of blocks must be an integer greater than 1')
        if key in self.dimsdict:
            from warnings import warn
            warn('this will replace dimension "%s" which already exists'%key)
            key = self.dimsdict[key]
            old_shape = self.shape
            try:
                temp = list(self.shape)
                temp[key] = num_blocks
                self.shape = tuple(temp)
                temp = list(self.dims)
                if block_names is None:
                    temp[key] = temp[key][0]
                    self.set_dims(temp)
                elif block_names == 'old':
                    if len(temp[key][1])!=num_blocks: raise ValueError('cannot retain old block names since new shape (%d) does not match the old shape (%d)'%(len(temp[key][1]), num_blocks))
                else:
                    temp[key] = (temp[key][0],  block_names)
                    self.set_dims(temp)
            except Exception as err:
                self.shape = old_shape
                raise err
            temp = list(self.omega)
            mean_omega = self.mean_omega()
            if len(mean_omega)>0: mean_omega = mean_omega[0]
            else: mean_omega = 0.0
            temp[key] = (mean_omega,)*num_blocks
            self.omega = tuple(temp)
            temp = list(self.pi)
            temp[key] = (1/num_blocks,)*num_blocks
            self.pi = tuple(temp)
            temp = list(self.rho)
            temp[key] = (1/num_blocks,)*num_blocks
            self.rho = tuple(temp)
        elif isinstance(key, int) or isinstance(key, str):
            if isinstance(key, str):
                keyname = key
                key = self.ndim
            else: keyname = None
            old_shape = self.shape
            try:                
                temp = list(self.shape)
                temp.insert(key, num_blocks)
                self.shape = tuple(temp)            
                temp = list(self.dims)
                if block_names is None: temp.insert(key, keyname)
                else: temp.insert(key, (keyname,  block_names))
                self.ndim += 1
                self.set_dims(temp)
            except Exception as err:
                self.ndim -=1
                self.shape = old_shape
                raise err
            temp = list(self.omega)
            mean_omega = self.mean_omega()
            if len(mean_omega)>0: mean_omega = mean_omega[0]
            else: mean_omega = 0.0
            temp.insert(key, (mean_omega,)*num_blocks)
            self.omega = tuple(temp)
            temp = list(self.pi)
            temp.insert(key, (1/num_blocks,)*num_blocks)
            self.pi = tuple(temp)
            temp = list(self.rho)
            temp.insert(key, (1/num_blocks,)*num_blocks)
            self.rho = tuple(temp)
        else: raise ValueError('either enter an integer index (dimension number) or a string key (dimension name)')
    
    def __getitem__(self, key):
        key = self.get_key(key)
        return {'index':key,
                'name':self.dims[key][0], 
                'blocks':self.dims[key][1], 
                'pi':self.pi[key], 
                'rho':self.rho[key], 
                'omega':self.omega[key]}
    
    def __setitem__(self, key, value):
        if self.iterable(key):
            if len(key)==2:
                k = self.get_key(key[0])
                if key[1]=='pi':
                    curr_pi = self.pi
                    curr_pi[k] = value
                    self.set_pi(curr_pi)
                elif key[1]=='rho':
                    curr_rho = self.rho
                    curr_rho[k] = value
                    self.set_rho(curr_rho)
                elif key[1]=='omega':
                    curr_omega = self.omega
                    curr_omega[k] = value
                    self.set_omega(curr_omega)
                elif key[1]=='name': raise ValueError('cannot set dim name by indexing; use method set_dims()')
                else: raise ValueError('no such property "%s"'%str(key[1]))
            else: raise ValueError('provide a (d, p) tuple to set property "p" of dimension "d"')
        else:
            raise ValueError('provide a (d, p) tuple to set property "p" of dimension "d"')
            
    def __delitem__(self, key):
        key = self.get_key(key)
        self.ndim -= 1
        temp = list(self.shape)
        temp.pop(key)
        self.shape = tuple(temp)
        temp = list(self.pi)
        pi = temp.pop(key)
        self.pi = tuple(temp)
        temp = list(self.rho)
        rho = temp.pop(key)
        self.rho = tuple(temp)
        temp = list(self.omega)
        omega = temp.pop(key)
        self.omega = tuple(temp)
        temp = list(self.dims)
        dims = temp.pop(key)
        self.dims = tuple(temp)
        self.dimsdict = dict(zip(temp, range(self.ndim)))
        dims_flat = [[(dim[0], d) for d in dim[1]] for dim in temp]
        from itertools import product
        self.dimsfull = tuple(product(*dims_flat))
        return {'index':key,
                'name':dims[0], 
                'blocks':dims[1], 
                'pi':pi, 
                'rho':rho, 
                'omega':omega}
        
    def __contains__(self, key):
        return key in self.dimsdict
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._curridx>=self.ndim:
            self._curridx = 0
            raise StopIteration
        else:
            self._curridx += 1
            return self[self._curridx-1]
        
    def __eq__(self, other):
        x = self.copy()
        y = other.copy()
        x.sort()
        y.sort()
        return x.pi==y.pi and x.rho==y.rho and x.omega==y.omega
     
    def __lt__(self, other):
        if self.ndim < other.ndim:
            x = self.copy()
            y = other.copy()
            x.sort()
            y.sort()
            i = 0
            j = 0
            while i<self.ndim:
                found = False
                while j<other.ndim:
                    if x.pi[i]==y.pi[j] and x.rho[i]==y.rho[j] and x.omega[i]==y.omega[j]:
                        found = True
                        break
                    else: j+=1
                if not found: return False
                else:
                    i+=1
            return True 
        else: return False
        
    def __le__(self, other):
        if self==other: return True
        else: return self<other
    
    def iterable(self, obj):
        if isinstance(obj, str): return False
        try:
            iter(obj)
            return True
        except TypeError: return False
    
    def sum_pi(self, pi=None, approx=False):
        if pi is None: pi = self.pi
        out = [sum(p) for p in pi]
        if approx: out = [round(x, self.precision) for x in out]
        return out
    
    def sum_rho(self, rho=None, approx=False):
        if rho is None: rho = self.rho
        out = [sum(r) for r in rho]
        if approx: out = [round(x, self.precision) for x in out]
        return out
    
    def dev_pi(self, pi=None, approx=False):
        if pi is None: pi = self.pi
        out = [sum([i**2 for i in p]) for p in pi]
        if approx: out = [round(x, self.precision) for x in out]
        return out
    
    def mean_rho(self, rho=None, pi=None, approx=False):
        if pi is None: pi = self.pi
        if rho is None: rho = self.rho
        out = [sum([m*n for m, n in zip(r, p)]) for r, p in zip(rho, pi)]
        if approx: out = [round(x, self.precision) for x in out]
        return out
    
    def mean_omega(self, omega=None, pi=None, approx=False):
        if pi is None: pi = self.pi
        if omega is None: omega = self.omega
        out = [sum([m*n for m, n in zip(o, p)]) for o, p in zip(omega, pi)]
        if approx: out = [round(x, self.precision) for x in out]
        return out
    
    def mean_homophily(self, rho=None, approx=False):
        return self.sum_rho(rho, approx)
    
    def mean_heterophily(self, rho=None, pi=None, approx=False):
        if pi is None: pi = self.pi
        if rho is None: rho = self.rho
        out = [sum([(1-m)*n/(1-n) for m, n in zip(r, p)]) for r, p in zip(rho, pi)]
        if approx: out = [round(x, self.precision) for x in out]
        return out
    
    def mean_homoffinity(self, omega=None, rho=None, approx=False):
        if rho is None: rho = self.rho
        if omega is None: omega = self.omega
        out = [sum([m*n for m, n in zip(o, r)]) for o, r in zip(omega, rho)]
        if approx: out = [round(x, self.precision) for x in out]
        return out
    
    def mean_heteroffinity(self, omega=None, rho=None, pi=None, approx=False):
        if pi is None: pi = self.pi
        if rho is None: rho = self.rho
        if omega is None: omega = self.omega
        out = [sum([(1-m)*n*w/(1-n) for w, m, n in zip(o, r, p)]) for o, r, p in zip(omega, rho, pi)]
        if approx: out = [round(x, self.precision) for x in out]
        return out

    def check_consistency(self, pi=None, omega=None, rho=None):
        if pi is None: pi = self.pi
        if omega is None: omega = self.omega
        if rho is None: rho = self.rho
        #pi tests
        if not all([all([0<i<1 for i in p])for p in pi]): raise ValueError('pi must be between 0 and 1 (exclusive)')
        if not all([p==1 for p in self.sum_pi(pi=pi, approx=True)]): raise ValueError('pi must sum up to 1')
        #omega tests
        if not all([all([0<=i for i in o])for o in omega]): raise ValueError('omega must be >=0')
        mean_omega = self.mean_omega(omega=omega, approx=True)
        for i in range(1, len(mean_omega)):
            if not mean_omega[i] == mean_omega[0]: raise ValueError('mean omega must remain same')
        #rho tests
        if not all([all([0<=i<=1 for i in r])for r in rho]): raise ValueError('rho must be between 0 and 1 (inclusive)')

    def get_nextdim(self):
        while str(self._currdim) in self.dimsdict:
            self._currdim += 1
        self._currdim += 1
        return str(self._currdim-1)
    
    def set_dims(self, dims=None):
        if dims is None: dims = [None for i in range(self.ndim)]
        if not self.iterable(dims): raise ValueError('provide all dimension names')
        dims = list(dims)
        if len(dims)!=self.ndim: raise ValueError('provide names for each dimension')
        dim_set = []
        for i in range(self.ndim):
            if self.iterable(dims[i]):
                found = False
                if len(dims[i])==2:
                    if (not self.iterable(dims[i][0])) and self.iterable(dims[i][1]):
                        if len(dims[i][1])==self.shape[i] and all([not self.iterable(j) for j in dims[i][1]]):
                            if len(set(dims[i][1]))!=self.shape[i]: raise ValueError('provide *unique* names for each block of dimension %d'%i)
                            else:
                                found = True
                                if dims[i][0] is None: dims[i] = (self.get_nextdim(), tuple(dims[i][1]))
                                else: dims[i] = (dims[i][0], tuple(dims[i][1]))
                        else: raise ValueError('provide names for each block of dimension %d'%i)
                if (not found) and len(dims[i])==self.shape[i] and all([not self.iterable(j) for j in dims[i]]):
                    if len(set(dims[i]))!=self.shape[i]: raise ValueError('provide *unique* names for each block of dimension %d'%i)
                    else:
                        found = True
                        dims[i] = (str(i), tuple(dims[i]))
                if not found: raise ValueError('provide names for each block of dimension %d'%i)
            else:
                if dims[i] is None: dims[i] = self.get_nextdim()
                dims[i] = (dims[i], tuple([str(j) for j in range(self.shape[i])]))
            if dims[i][0] in dim_set: raise ValueError('provide *unique* name for each dimension; clash on "%s"'%str(dims[i][0]))
            else: dim_set.append(dims[i][0])        
        if any([not isinstance(dim, str) for dim in dim_set]): raise TypeError('dimension names must be strings')
        self.dims = tuple(dims)
        self.dimsdict = dict(zip(dim_set, range(self.ndim)))
        dims_flat = [[(dim[0]+':'+str(d)) for d in dim[1]] for dim in dims]
        from itertools import product
        self.dimsfull = tuple([','.join(i) for i in product(*dims_flat)])

    def set_pi(self, pi=None):
        if pi is None: pi = tuple([(1/b,)*b for b in self.shape])
        if not self.iterable(pi): pi = tuple([(float(pi),)*b for b in self.shape])
        pi = list(pi)
        if all([not self.iterable(p) for p in pi]) and self.ndim==1: pi = [pi]
        if len(pi)!=self.ndim: raise ValueError('provide pi for every dimension')
        for i in range(self.ndim):
            if not self.iterable(pi[i]): pi[i] = (pi[i],)*self.shape[i]
            if len(pi[i])!=self.shape[i]: raise ValueError('provide pi for every block of dimension %d'%i)
            else: pi[i] = tuple(pi[i])
        pi = tuple(pi)
        self.check_consistency(pi=pi)
        self.pi = pi
        
    def set_omega(self, omega=0.):
        if not self.iterable(omega): omega = tuple([(float(omega),)*b for b in self.shape])
        omega = list(omega)
        if all([not self.iterable(o) for o in omega]) and self.ndim==1: omega = [omega]
        if len(omega)!=self.ndim: raise ValueError('provide omega for every dimension')
        for i in range(self.ndim):
            if not self.iterable(omega[i]): omega[i] = (omega[i],)*self.shape[i]
            if len(omega[i])!=self.shape[i]: raise ValueError('provide omega for every block of dimension %d'%i)
            else: omega[i] = tuple(omega[i])
        omega = tuple(omega)
        self.check_consistency(omega=omega)
        self.omega = omega
        
    def set_rho(self, rho=None):
        if rho is None: rho = tuple([(1/b,)*b for b in self.shape])
        if not self.iterable(rho): rho = tuple([(float(rho),)*b for b in self.shape])
        rho = list(rho)
        if all([not self.iterable(r) for r in rho]) and self.ndim==1: rho = [rho]
        if len(rho)!=self.ndim: raise ValueError('provide rho for every dimension')
        for i in range(self.ndim):
            if not self.iterable(rho[i]): rho[i] = (rho[i],)*self.shape[i]
            if len(rho[i])!=self.shape[i]: raise ValueError('provide rho for every block of dimension %d'%i)
            else: rho[i] = tuple(rho[i])
        rho = tuple(rho)
        self.check_consistency(rho=rho)
        self.rho = rho
    
    class StochasticBlockModel():
        
        def __init__(self, egocentric_sbm, model_type='full', model_name='sbm'):
            
            if model_type not in ['full', 'pp', 'ppcollapsed']: raise ValueError('invalid model type "%s"'%model_type)
            self.type = model_type
            self.name = model_name
            self.dims = egocentric_sbm.dims
            self.dimsfull = egocentric_sbm.dimsfull
            
            import numpy as np
            
            def kron(list_of_mats):
                a = list_of_mats[0].copy()
                for i in range(1, len(list_of_mats)):
                    a = np.kron(a, list_of_mats[i])
                return a
            
            if model_type=='full':
                self.params = [[(o, r/p, (1-r)/(1-p)) for r, o, p in zip(egocentric_sbm.rho[i], egocentric_sbm.omega[i], egocentric_sbm.pi[i])] for i in range(egocentric_sbm.ndim)]
                self.matrix = [(np.diag([x[0] for x in item]), np.diag([x[1]-x[2] for x in item])+np.vstack([x[2]*np.ones(len(item)) for x in item])) for item in self.params]
                self.matrixfull = (kron([x[0] for x in self.matrix]), kron([x[1] for x in self.matrix]))
            elif model_type=='pp': #planted-partition model
                self.params = (egocentric_sbm.mean_omega()[0], list(zip(egocentric_sbm.mean_homophily(), egocentric_sbm.mean_heterophily())))
                self.matrix = [(self.params[0]*np.eye(egocentric_sbm.shape[i]), (self.params[1][i][0]-self.params[1][i][1])*np.eye(egocentric_sbm.shape[i])+self.params[1][i][1]*np.ones([egocentric_sbm.shape[i], egocentric_sbm.shape[i]])) for i in range(egocentric_sbm.ndim)]
                self.matrixfull = (kron([x[0] for x in self.matrix]), kron([x[1] for x in self.matrix]))
            elif model_type=='ppcollapsed':
                self.params = list(zip(egocentric_sbm.mean_homoffinity(), egocentric_sbm.mean_heteroffinity()))
                self.matrix = [(self.params[i][0]-self.params[i][1])*np.eye(egocentric_sbm.shape[i])+self.params[i][1]*np.ones([egocentric_sbm.shape[i], egocentric_sbm.shape[i]]) for i in range(egocentric_sbm.ndim)]
                self.matrixfull = kron(self.matrix)
    
    def get_model(self, model_type='full', model_name='sbm'):
        return self.StochasticBlockModel(self, model_type, model_name)

In [426]:
d3.sort()

[(2, 0.8, 0.0, 0.8, 0.2, 0.0, 1), (2, 0.8, 0.0, 0.9, 0.2, 0.0, 0)]


([1, 0], [[1, 0], [0, 1]])

In [425]:
d3.set_pi([[0.2,0.8],[0.8,0.2]])
d3.set_rho([[1,0.8],[0.9,0]])

In [459]:
d1 = EgocentricSBM([2,4])
d2 = EgocentricSBM([2,3])
d3 = EgocentricSBM([2,2])

In [469]:
d1.set_pi([[0.5,0.5],[0.2,0.2,0.4,0.2]])
d1.set_omega([[4, 6], 5])
d1.set_rho((1,[1,0.5,0.1,0.5]))

In [471]:
d5 <= d1

False

In [461]:
d5 = d1.copy()

In [464]:
del d5[0]

In [437]:
d1 == d5

True

In [392]:
d1 == d5

True

In [279]:
del d1[0]

In [281]:
print(d5)

name:	egocentric_sbm
ndim:	2
shape:	(2, 4)
----	------
dims	blocks
----	------
0	(0, 1)
1	(2, 3, 4, 5)
----	------
pi:	((0.8, 0.2), (0.4, 0.3, 0.2, 0.1))
rho:	((0.5, 0.5), (0.25, 0.25, 0.25, 0.25))
omega:	((5.0, 5.0), (5.0, 5.0, 5.0, 5.0))


In [151]:
dict(zip(range(len(d1)), iter(d1)))

{0: {'index': 0,
  'name': '0',
  'blocks': ('0', '1'),
  'pi': (0.8, 0.2),
  'rho': (0.5, 0.5),
  'omega': (5.0, 5.0)},
 1: {'index': 1,
  'name': '1',
  'blocks': ('0', '1', '2', '3'),
  'pi': (0.4, 0.3, 0.2, 0.1),
  'rho': (0.25, 0.25, 0.25, 0.25),
  'omega': (5.0, 5.0, 5.0, 5.0)}}

In [163]:
g=iter(d1)

In [166]:
next(g)

StopIteration: 

In [167]:
print(d1)

name:	egocentric_sbm
ndim:	2
shape:	(2, 4)
----	------
dims	blocks
----	------
0	('0', '1')
1	('0', '1', '2', '3')
----	------
pi:	((0.8, 0.2), (0.4, 0.3, 0.2, 0.1))
rho:	((0.5, 0.5), (0.25, 0.25, 0.25, 0.25))
omega:	((5.0, 5.0), (5.0, 5.0, 5.0, 5.0))


In [170]:
from itertools import cycle

In [171]:
j=cycle(d1)

In [172]:
j

In [178]:
next(j)

{'index': 1,
 'name': '1',
 'blocks': ('0', '1', '2', '3'),
 'pi': (0.4, 0.3, 0.2, 0.1),
 'rho': (0.25, 0.25, 0.25, 0.25),
 'omega': (5.0, 5.0, 5.0, 5.0)}

In [179]:
print(d1)

name:	egocentric_sbm
ndim:	2
shape:	(2, 4)
----	------
dims	blocks
----	------
0	('0', '1')
1	('0', '1', '2', '3')
----	------
pi:	((0.8, 0.2), (0.4, 0.3, 0.2, 0.1))
rho:	((0.5, 0.5), (0.25, 0.25, 0.25, 0.25))
omega:	((5.0, 5.0), (5.0, 5.0, 5.0, 5.0))


In [185]:
hash(d1)

-9223371895660953333

In [188]:
print(d1,d2)

name:	egocentric_sbm
ndim:	2
shape:	(2, 4)
----	------
dims	blocks
----	------
0	('0', '1')
1	('0', '1', '2', '3')
----	------
pi:	((0.5, 0.5), (0.25, 0.25, 0.25, 0.25))
rho:	((0.5, 0.5), (0.25, 0.25, 0.25, 0.25))
omega:	((0.0, 0.0), (0.0, 0.0, 0.0, 0.0)) name:	egocentric_sbm
ndim:	2
shape:	(2, 3)
----	------
dims	blocks
----	------
0	('0', '1')
1	('0', '1', '2')
----	------
pi:	((0.5, 0.5), (0.3333333333333333, 0.3333333333333333, 0.3333333333333333))
rho:	((0.5, 0.5), (0.3333333333333333, 0.3333333333333333, 0.3333333333333333))
omega:	((0.0, 0.0), (0.0, 0.0, 0.0))


In [190]:
hash({1,2,3})

TypeError: unhashable type: 'set'

In [201]:
print(d3)

name:	egocentric_sbm
ndim:	2
shape:	(2, 4)
----	------
dims	blocks
----	------
0	('0', '1')
1	('0', '1', '2', '3')
----	------
pi:	((0.5, 0.5), (0.25, 0.25, 0.25, 0.25))
rho:	((0.5, 0.5), (0.25, 0.25, 0.25, 0.25))
omega:	((0.0, 0.0), (0.0, 0.0, 0.0, 0.0))


In [196]:
type(d1)

__main__.EgocentricSBM

In [206]:
d3 == d1

True

In [209]:
d3[(0, 'pi')] = [0.8,0.2]

AttributeError: 'tuple' object has no attribute 'copy'

In [208]:
copy((1,2,3))

The system cannot find the file specified.


In [222]:
{1,2} <= {1,2,3,4}

True

In [224]:
(1,3) < (1,2,3,4,5)

False

In [225]:
d1<d2

TypeError: '<' not supported between instances of 'EgocentricSBM' and 'EgocentricSBM'

In [226]:
d1 in d3

TypeError: unhashable type: 'EgocentricSBM'

In [228]:
[print(i) for i in zip([1,2,3],{4,5,6})]

(1, 4)
(2, 5)
(3, 6)


[None, None, None]

In [229]:
g=set()

In [232]:
g.add(4)

In [233]:
g

{3, 4}

In [234]:
g=[1,2,3]

In [235]:
g.top()

AttributeError: 'list' object has no attribute 'top'

In [258]:
from copy import copy as cp

In [266]:
d4 = cp(d1)

In [269]:
print(d4)

name:	egocentric_sbm
ndim:	2
shape:	(2, 4)
----	------
dims	blocks
----	------
0	(0, 1)
1	(2, 3, 4, 5)
----	------
pi:	((0.8, 0.2), (0.4, 0.3, 0.2, 0.1))
rho:	((0.5, 0.5), (0.25, 0.25, 0.25, 0.25))
omega:	((5.0, 5.0), (5.0, 5.0, 5.0, 5.0))


In [268]:
d1.pop()

{'index': 1,
 'name': '1',
 'blocks': (2, 3, 4, 5),
 'pi': (0.4, 0.3, 0.2, 0.1),
 'rho': (0.25, 0.25, 0.25, 0.25),
 'omega': (5.0, 5.0, 5.0, 5.0)}

In [270]:
print(d1)

name:	egocentric_sbm
ndim:	1
shape:	(2,)
----	------
dims	blocks
----	------
0	(0, 1)
----	------
pi:	((0.8, 0.2),)
rho:	((0.5, 0.5),)
omega:	((5.0, 5.0),)


In [287]:
f=(5,2,3)

In [290]:
a, b = sorted(f, reverse=True)

ValueError: too many values to unpack (expected 2)

In [295]:
sorted((zip((1,1),(3,3),(5,6))))

[(1, 3, 5), (1, 3, 6)]

In [399]:
gggg=chain((1,2,3),(3,4,6))

In [394]:
from itertools import chain

In [398]:
tuple(chain(*gggg))

(1, 2, 3, 3, 4, 6)

In [400]:
tuple(gggg)

(1, 2, 3, 3, 4, 6)